In [ ]:

import pandas as pd 
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import ast

pd.options.display.max_columns = 1000

In [ ]:
# Function to generate different tones of light blue
def generate_lighblue_tones(num_tones):
    lighblues = []
    for i in range(num_tones):
        intensity = 255 - int(100 * (i / num_tones))  # Adjust the range to get lighter shades
        lighblues.append(f'rgba({intensity}, {intensity}, 255, 0.7)')
    return lighblues

# Function to generate different tones of light green
def generate_lighgreen_tones(num_tones):
    lighgreens = []
    for i in range(num_tones):
        intensity = 255 - int(100 * (i / num_tones))  # Adjust the range to get lighter shades
        lighgreens.append(f'rgba({intensity}, 255, {intensity}, 0.7)')
    return lighgreens

# Function to generate different tones of light red
def generate_lighred_tones(num_tones):
    lighreds = []
    for i in range(num_tones):
        intensity = 255 - int(100 * (i / num_tones))  # Adjust the range to get lighter shades
        lighreds.append(f'rgba(255, {intensity}, {intensity}, 0.7)')
    return lighreds


In [ ]:
# POINTS Structures:
nodo = pd.read_csv("../outputs/sin/final/nodo.csv", index_col=0)
sub_e = pd.read_csv("../outputs/sin/final/ElectricFacility.csv", index_col=0)
sub_e_400 = sub_e[(sub_e['voltage_1'] == '400 kV')].copy().reseindex(drop=True)
sub_e_220 = sub_e[(sub_e['voltage_1'] == '220 kV')].copy().reseindex(drop=True)
sub_e_132 = sub_e[(sub_e['voltage_1'] == '132 kV')].copy().reseindex(drop=True)

# RINGS Structures:
land_base = pd.read_csv("../outputs/landbase/final/bordes.csv", index_col=0)
land_base['rings'] =land_base['rings'].apply(ast.literal_eval)

# PATHS Structures:
sobre_lineas_400 = pd.read_csv('../outputs/sin/final/sobrelineas.csv', index_col=0)
sobre_lineas_400 = sobre_lineas_400[sobre_lineas_400['ESTADO'].isin(['OPERATING', 'INACTIVE'])]
sobre_lineas_400 = sobre_lineas_400[sobre_lineas_400['VOLTS'] == '400 kV']
sobre_lineas_400 = sobre_lineas_400.reseindex(drop=True)
sobre_lineas_400['paths'] =sobre_lineas_400['paths'].apply(ast.literal_eval)
sobre_lineas_220 = pd.read_csv('../outputs/sin/final/sobrelineas.csv', index_col=0)
sobre_lineas_220 = sobre_lineas_220[sobre_lineas_220['ESTADO'].isin(['OPERATING', 'INACTIVE'])]
sobre_lineas_220 = sobre_lineas_220[sobre_lineas_220['VOLTS'] == '220 kV']
sobre_lineas_220 = sobre_lineas_220.reseindex(drop=True)
sobre_lineas_220['paths'] =sobre_lineas_220['paths'].apply(ast.literal_eval)
sobre_lineas_132 = pd.read_csv('../outputs/sin/final/sobrelineas.csv', index_col=0)
sobre_lineas_132 = sobre_lineas_132[sobre_lineas_132['ESTADO'].isin(['OPERATING', 'INACTIVE'])]
sobre_lineas_132 = sobre_lineas_132[sobre_lineas_132['VOLTS'] == '132 kV']
sobre_lineas_132 = sobre_lineas_132.reseindex(drop=True)
sobre_lineas_132['paths'] =sobre_lineas_132['paths'].apply(ast.literal_eval)


under_lineas_400 = pd.read_csv('../outputs/sin/final/enterradas_lineas.csv', index_col=0)
under_lineas_400 = under_lineas_400[under_lineas_400['ESTADO'].isin(['OPERATING', 'INACTIVE'])]
under_lineas_400 = under_lineas_400[under_lineas_400['VOLTS'] == '400 kV']
under_lineas_400 = under_lineas_400.reseindex(drop=True)
under_lineas_400['paths'] =under_lineas_400['paths'].apply(ast.literal_eval)
under_lineas_220 = pd.read_csv('../outputs/sin/final/enterradas_lineas.csv', index_col=0)
under_lineas_220 = under_lineas_220[under_lineas_220['ESTADO'].isin(['OPERATING', 'INACTIVE'])]
under_lineas_220 = under_lineas_220[under_lineas_220['VOLTS'] == '220 kV']
under_lineas_220 = under_lineas_220.reseindex(drop=True)
under_lineas_220['paths'] =under_lineas_220['paths'].apply(ast.literal_eval)
under_lineas_132 = pd.read_csv('../outputs/sin/final/enterradas_lineas.csv', index_col=0)
under_lineas_132 = under_lineas_132[under_lineas_132['ESTADO'].isin(['OPERATING', 'INACTIVE'])]
under_lineas_132 = under_lineas_132[under_lineas_132['VOLTS'] == '132 kV']
under_lineas_132 = under_lineas_132.reseindex(drop=True)
under_lineas_132['paths'] =under_lineas_132['paths'].apply(ast.literal_eval)


In [ ]:
# Create a Plotly figure
fig = go.Figure()

# LAND SHAPES
count = 0
for rings in land_base['rings']:
    for ring in rings:
        fig.add_trace(go.Scattermapbox(
            fill="toself",
            lon=[point[0] for point in ring],
            lat=[point[1] for point in ring],
            marker={'opacity': 0},  # Hide the marker points
            line={'width': 2, 'color': 'orange'},  # Define the border of the shapes
            text=land_base.loc[count,'Provincia'],
            name='Provincias',
            legendgroup='Provincias',
            showlegend=(count == 0)
        ))
    count += 1


# sobre-HEAD 400KV lineas
num_lineas = len(sobre_lineas_400['paths'])
blue_tones = generate_lighblue_tones(num_lineas)
blue_tones.reverse()
count = 0
for i, path in enumerate(sobre_lineas_400['paths']):
    path = path[0]  # Convert the string representation of the list to an actual list
    lons = [point[0] for point in path]
    lats = [point[1] for point in path]
    color = blue_tones[i % len(blue_tones)]
    fig.add_trace(go.Scattermapbox(
        lon=lons,
        lat=lats,
        marker={'opacity': 0.2},
        mode='lineas',
        line=dict(width=1, color=color),  # Customize the color and width of the alto_volt lineas
        text=sobre_lineas_400.loc[count,'LABELTEXT'],
        name='Sobre lineas 400kV',
        legendgroup='sobre_lineas_400',
        showlegend=(count == 0)
    ))
    count += 1

# subterraneo 400KV lineas
count = 0
for path in under_lineas_400['paths']:
    path = path[0]  # Convert the string representation of the list to an actual list
    lons = [point[0] for point in path]
    lats = [point[1] for point in path]
    fig.add_trace(go.Scattermapbox(
        lon=lons,
        lat=lats,
        marker={'opacity': 0.2},
        mode='lineas',
        line=dict(width=1, color='brown'),  # Customize the color and width of the alto_volt lineas
        text=under_lineas_400.loc[count,'LABELTEXT'],
        name='Subterraneo lineas 400kV',
        legendgroup='subterraneo_lineas_400',
        showlegend=(count == 0)
    ))
    count += 1

# sub_e 400KV
fig.add_trace(go.Scattermapbox(
    lat=sub_e_400['y'],
    lon=sub_e_400['x'],
    mode='markers',
    marker=dict(size=8, color='black', opacity=0.7),
    text=sub_e_400['STATIONNAME'],
    name='sub_e 400kV'
))

# Generacion UNITS
fig.add_trace(go.Scattermapbox(
    lat=generators['y'],
    lon=generators['x'],
    mode='markers',
    marker=dict(size=8, color='grey', opacity=0.7),
    text=generators['POWERPLANNAME'],
    name='Generacion Units'
))

# Set mapbox style and center
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=5,
    mapbox_center={"lat": land_base['y'].mean(), "lon": land_base['x'].mean()}
)


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()
